In [1]:
import torch
from torch.nn import Sequential, Linear, ReLU, Sigmoid, Tanh, Dropout, Upsample
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import NNConv
from torch_geometric.nn import GCNConv
from torch_geometric.nn import BatchNorm
import numpy as np
from torch_geometric.data import Data
from torch.autograd import Variable
import networkx as nx

import os.path as osp
import pickle
from scipy.linalg import sqrtm
import argparse
from scipy.stats import wasserstein_distance
from torch.distributions import normal, kl


import argparse
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, GAE, VGAE, InnerProductDecoder, ARGVA
from torch_geometric.utils import train_test_split_edges
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import KFold
import pandas as pd


In [2]:
from data_preparation import load_data_tensor

lr_train, lr_test, hr_train = load_data_tensor("dgl-icl")



In [3]:
# Number of subjects in simulated data 
N_SUBJECTS = 167

# Number of ROIs in source brain graph for simulated data 
N_SOURCE_NODES = 160

# Number of ROIs in target brain graph for simulated data
N_TARGET_NODES = 268

# Number of traning epochs
N_EPOCHS = 10


####** DO NOT MODIFY BELOW **####
N_SOURCE_NODES_F =int((N_SOURCE_NODES*(N_SOURCE_NODES-1))/2)
N_TARGET_NODES_F =int((N_TARGET_NODES*(N_TARGET_NODES-1))/2)
###**************************####

In [4]:
class Aligner(torch.nn.Module):
    def __init__(self):
        
        super(Aligner, self).__init__()

        nn = Sequential(Linear(1, N_SOURCE_NODES*N_SOURCE_NODES), ReLU())
        self.conv1 = NNConv(N_SOURCE_NODES, N_SOURCE_NODES, nn, aggr='mean', root_weight=True, bias=True)
        self.conv11 = BatchNorm(N_SOURCE_NODES, eps=1e-03, momentum=0.1, affine=True, track_running_stats=True)

        nn = Sequential(Linear(1, N_SOURCE_NODES), ReLU())
        self.conv2 = NNConv(N_SOURCE_NODES, 1, nn, aggr='mean', root_weight=True, bias=True)
        self.conv22 = BatchNorm(1, eps=1e-03, momentum=0.1, affine=True, track_running_stats=True)

        nn = Sequential(Linear(1, N_SOURCE_NODES), ReLU())
        self.conv3 = NNConv(1, N_SOURCE_NODES, nn, aggr='mean', root_weight=True, bias=True)
        self.conv33 = BatchNorm(N_SOURCE_NODES, eps=1e-03, momentum=0.1, affine=True, track_running_stats=True)


    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.pos_edge_index, data.edge_attr

        x1 = F.sigmoid(self.conv11(self.conv1(x, edge_index, edge_attr)))
        x1 = F.dropout(x1, training=self.training)

        x2 = F.sigmoid(self.conv22(self.conv2(x1, edge_index, edge_attr)))

        x2 = F.dropout(x2, training=self.training)

        x3 = torch.cat([F.sigmoid(self.conv33(self.conv3(x2, edge_index, edge_attr))), x1], dim=1)
        # x3 = torch.cat([F.sigmoid(self.conv3(x2, edge_index, edge_attr)), x1], dim=1)
        x4 = x3[:, 0:N_SOURCE_NODES]
        x5 = x3[:, N_SOURCE_NODES:2*N_SOURCE_NODES]

        x6 = (x4 + x5) / 2
        return x6








class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        nn = Sequential(Linear(1, N_SOURCE_NODES*N_SOURCE_NODES),ReLU())
        self.conv1 = NNConv(N_SOURCE_NODES, N_SOURCE_NODES, nn, aggr='mean', root_weight=True, bias=True)
        self.conv11 = BatchNorm(N_SOURCE_NODES, eps=1e-03, momentum=0.1, affine=True, track_running_stats=True)

        nn = Sequential(Linear(1, N_TARGET_NODES*N_SOURCE_NODES), ReLU())
        self.conv2 = NNConv(N_TARGET_NODES, N_SOURCE_NODES, nn, aggr='mean', root_weight=True, bias=True)
        self.conv22 = BatchNorm(N_SOURCE_NODES, eps=1e-03, momentum=0.1, affine=True, track_running_stats=True)

        nn = Sequential(Linear(1, N_TARGET_NODES*N_SOURCE_NODES), ReLU())
        self.conv3 = NNConv(N_SOURCE_NODES, N_TARGET_NODES, nn, aggr='mean', root_weight=True, bias=True)
        self.conv33 = BatchNorm(N_TARGET_NODES, eps=1e-03, momentum=0.1, affine=True, track_running_stats=True)


        # self.layer= torch.nn.ConvTranspose2d(N_TARGET_NODES, N_TARGET_NODES,5)


    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.pos_edge_index, data.edge_attr
        # x = torch.squeeze(x)

        x1 = F.sigmoid(self.conv11(self.conv1(x, edge_index, edge_attr)))
        x1 = F.dropout(x1, training=self.training)

        # x2 = F.sigmoid(self.conv22(self.conv2(x1, edge_index, edge_attr)))
        # x2 = F.dropout(x2, training=self.training)

        x3 = F.sigmoid(self.conv33(self.conv3(x1, edge_index, edge_attr)))
        x3 = F.dropout(x3, training=self.training)



        x4  = torch.matmul(x3.t(), x3)

        return x4

class Discriminator(torch.nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = GCNConv(N_TARGET_NODES, N_TARGET_NODES, cached=True)
        self.conv2 = GCNConv(N_TARGET_NODES, 1, cached=True)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.pos_edge_index, data.edge_attr
        x = torch.squeeze(x)
        x1 = F.sigmoid(self.conv1(x, edge_index))
        x1 = F.dropout(x1, training=self.training)
        x2 = F.sigmoid(self.conv2(x1, edge_index))
        #         # x2 = F.dropout(x2, training=self.training)


        return x2


In [5]:
# put it back into a 2D symmetric array


def topological_measures(data):
    # ROI is the number of brain regions (i.e.,35 in our case)
    ROI = 160

    topology = []



    # A = to_2d(data)
    np.fill_diagonal(data, 0)

    # create a graph from similarity matrix
    G = nx.from_numpy_matrix(np.absolute(data))
    U = G.to_undirected()

    # Centrality #

    # compute closeness centrality and transform the output to vector
    cc = nx.closeness_centrality(U, distance="weight")
    closeness_centrality = np.array([cc[g] for g in U])
    # compute betweeness centrality and transform the output to vector
    # bc = nx.betweenness_centrality(U, weight='weight')
    # bc = (nx.betweenness_centrality(U))
    betweenness_centrality = np.array([cc[g] for g in U])
    # # compute egeinvector centrality and transform the output to vector
    ec = nx.eigenvector_centrality_numpy(U)
    eigenvector_centrality = np.array([ec[g] for g in U])


    topology.append(closeness_centrality)  # 0
    topology.append(betweenness_centrality)  # 1
    topology.append(eigenvector_centrality)  # 2

    return topology
# put it back into a 2D symmetric array

def eigen_centrality(data):
    # ROI is the number of brain regions (i.e.,35 in our case)
    ROI = 268

    topology_eigen = []


    data = data.squeeze()
    # A = to_2d(data)
    np.fill_diagonal(data, 0)

    # create a graph from similarity matrix
    G = nx.from_numpy_array(np.absolute(data))
    U = G.to_undirected()

    # Centrality #


    # # compute egeinvector centrality and transform the output to vector
    ec = nx.eigenvector_centrality_numpy(U)
    eigenvector_centrality = np.array([ec[g] for g in U])



    topology_eigen.append(eigenvector_centrality)  # 2

    return topology_eigen

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("running on GPU")
else:
    device = torch.device("cpu")
    print("running on CPU")

l1_loss = torch.nn.L1Loss()
adversarial_loss = torch.nn.BCELoss()
adversarial_loss.to(device)
l1_loss.to(device)


def pearson_coor(input, target):
    vx = input - torch.mean(input)
    vy = target - torch.mean(target)
    cost = torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))
    return cost


def GT_loss(target, predicted):

    # l1_loss
    loss_pix2pix = l1_loss(target, predicted)

    # topological_loss
    target_n = target.detach().cpu().clone().numpy()
    predicted_n = predicted.detach().cpu().clone().numpy()
    torch.cuda.empty_cache()

    target_t = eigen_centrality(target_n)
    real_topology = torch.tensor(target_t)
    predicted_t = eigen_centrality(predicted_n)
    fake_topology = torch.tensor(predicted_t)
    topo_loss = l1_loss(fake_topology, real_topology)

    pc_loss = pearson_coor(target, predicted).to(device)
    torch.cuda.empty_cache()

    G_loss = loss_pix2pix + (1 - pc_loss) + topo_loss

    return G_loss


def Alignment_loss(target, predicted):
    # l_loss1 = torch.abs(nn.KLDivLoss()(F.softmax(zt1), F.softmax(z_s1.t())))

    kl_loss = torch.abs(F.kl_div(F.softmax(target), F.softmax(predicted), None, None, 'sum'))
    kl_loss = (1/350) * kl_loss
    return kl_loss

running on GPU


In [7]:
import numpy as np
from scipy.io import loadmat
from torch_geometric.data import Data
import torch



def convert_vector_to_graph_FC(data):
    """
        convert subject vector to adjacency matrix then use it to create a graph
        edge_index:
        edge_attr:
        x:
    """

    data.reshape(1, N_TARGET_NODES_F)
    # create adjacency matrix
    tri = np.zeros((N_TARGET_NODES, N_TARGET_NODES))
    tri[np.triu_indices(N_TARGET_NODES, 1)] = data
    tri = tri + tri.T
    tri[np.diag_indices(N_TARGET_NODES)] = 1

    edge_attr = torch.Tensor(tri).view(N_TARGET_NODES**2, 1)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    counter = 0
    pos_counter = 0
    neg_counter = 0
    N_ROI = N_TARGET_NODES

    pos_edge_index = torch.zeros(2, N_ROI * N_ROI)
    neg_edge_indexe = []
    # pos_edge_indexe = []
    for i in range(N_ROI):
        for j in range(N_ROI):
            pos_edge_index[:, counter] = torch.tensor([i, j])
            counter += 1

        # xx = torch.ones(160, 160, dtype=torch.float)

        x = torch.tensor(tri, dtype=torch.float)
        pos_edge_index = torch.tensor(pos_edge_index, dtype=torch.long)


    return Data(x=x, pos_edge_index=pos_edge_index, edge_attr=edge_attr)

def convert_vector_to_graph_RH(data):
    """
        convert subject vector to adjacency matrix then use it to create a graph
        edge_index:
        edge_attr:
        x:
    """

    data.reshape(1, N_SOURCE_NODES_F)
    # create adjacency matrix
    tri = np.zeros((N_SOURCE_NODES, N_SOURCE_NODES))
    tri[np.triu_indices(N_SOURCE_NODES, 1)] = data
    tri = tri + tri.T
    tri[np.diag_indices(N_SOURCE_NODES)] = 1

    edge_attr = torch.Tensor(tri).view(N_SOURCE_NODES**2, 1)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    counter = 0
    pos_counter = 0
    neg_counter = 0
    N_ROI = N_SOURCE_NODES

    pos_edge_index = torch.zeros(2, N_ROI * N_ROI)
    neg_edge_indexe = []
    # pos_edge_indexe = []
    for i in range(N_ROI):
        for j in range(N_ROI):
            pos_edge_index[:, counter] = torch.tensor([i, j])
            counter += 1

        # xx = torch.ones(160, 160, dtype=torch.float)

        x = torch.tensor(tri, dtype=torch.float)
        pos_edge_index = torch.tensor(pos_edge_index, dtype=torch.long)

        return Data(x=x, pos_edge_index=pos_edge_index, edge_attr=edge_attr)


def cast_data_vector_RH(dataset):
    """
        convert subject vectors to graph and append it in a list
    """

    dataset_g = []

    for subj in range(dataset.shape[0]):
        dataset_g.append(convert_vector_to_graph_RH(dataset[subj]))

    return dataset_g

def cast_data_vector_FC(dataset):
    """
        convert subject vectors to graph and append it in a list
    """

    dataset_g = []

    for subj in range(dataset.shape[0]):
        dataset_g.append(convert_vector_to_graph_FC(dataset[subj]))

def convert_vector_to_graph_RH_2(data):
    """
        convert subject vector to adjacency matrix then use it to create a graph
        edge_index:
        edge_attr:
        x:
    """
    tri = data

    edge_attr = torch.Tensor(tri).view(N_SOURCE_NODES**2, 1)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    counter = 0
    pos_counter = 0
    neg_counter = 0
    N_ROI = N_SOURCE_NODES

    pos_edge_index = torch.zeros(2, N_ROI * N_ROI)
    neg_edge_indexe = []
    # pos_edge_indexe = []
    for i in range(N_ROI):
        for j in range(N_ROI):
            pos_edge_index[:, counter] = torch.tensor([i, j])
            counter += 1

        # xx = torch.ones(160, 160, dtype=torch.float)

        x = torch.tensor(tri, dtype=torch.float)
        pos_edge_index = torch.tensor(pos_edge_index, dtype=torch.long)

        return Data(x=x, pos_edge_index=pos_edge_index, edge_attr=edge_attr)


#Copied the above functions and adapted the beginning by deleting the part where they
#transformed the vector to a matrix
def convert_vector_to_graph_FC_2(data):
    """
        convert subject vector to adjacency matrix then use it to create a graph
        edge_index:
        edge_attr:
        x:
    """
    tri = data
    edge_attr = torch.Tensor(tri).view(N_TARGET_NODES**2, 1)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    counter = 0
    pos_counter = 0
    neg_counter = 0
    N_ROI = N_TARGET_NODES

    pos_edge_index = torch.zeros(2, N_ROI * N_ROI)
    neg_edge_indexe = []
    # pos_edge_indexe = []
    for i in range(N_ROI):
        for j in range(N_ROI):
            pos_edge_index[:, counter] = torch.tensor([i, j])
            counter += 1

        # xx = torch.ones(160, 160, dtype=torch.float)

        x = torch.tensor(tri, dtype=torch.float)
        pos_edge_index = torch.tensor(pos_edge_index, dtype=torch.long)


    return Data(x=x, pos_edge_index=pos_edge_index, edge_attr=edge_attr)


def cast_data_vector_RH_2(dataset):
    """
        convert subject vectors to graph and append it in a list
    """

    dataset_g = []

    for subj in range(dataset.shape[0]):
        dataset_g.append(convert_vector_to_graph_RH_2(dataset[subj]))

    return dataset_g

def cast_data_vector_FC_2(dataset):
    """
        convert subject vectors to graph and append it in a list
    """

    dataset_g = []

    for subj in range(dataset.shape[0]):
        dataset_g.append(convert_vector_to_graph_FC_2(dataset[subj]))
    return dataset_g



def convert_generated_to_graph(data):
    """
        convert generated output from G to a graph
    """

    dataset = []

# for data in data1:
    counter = 0
    N_ROI = N_TARGET_NODES
    pos_edge_index = torch.zeros(2, N_ROI * N_ROI, dtype=torch.long)
    for i in range(N_ROI):
        for j in range(N_ROI):
            pos_edge_index[:, counter] = torch.tensor([i, j])
            counter += 1

    x = data
    pos_edge_index = torch.tensor(pos_edge_index, dtype=torch.long)
    data = Data(x=x, pos_edge_index= pos_edge_index, edge_attr=data.view(N_TARGET_NODES**2, 1))
    dataset.append(data)

    return dataset

def convert_generated_to_graph_Al(data1):
    """
        convert generated output from G to a graph
    """

    dataset = []

    # for data in data1:
    counter = 0
    N_ROI = N_SOURCE_NODES
    pos_edge_index = torch.zeros(2, N_ROI * N_ROI, dtype=torch.long)
    for i in range(N_ROI):
        for j in range(N_ROI):
            pos_edge_index[:, counter] = torch.tensor([i, j])
            counter += 1

    # x = data
    pos_edge_index = torch.tensor(pos_edge_index, dtype=torch.long)
    data = Data(x=data1, pos_edge_index=pos_edge_index, edge_attr=data1.view(N_SOURCE_NODES*N_SOURCE_NODES, 1))
    dataset.append(data)

    return dataset

import torch_geometric
def convert_func_data_to_graph_list(data, num_nodes=160, h=True):
    batch_size = data.shape[0]
    dataset = []# Create an array to store the adjacency matrices

    for i in range(batch_size):
        x = data[i]
        dense_adj = torch.ones((num_nodes, num_nodes))

        # Convert the dense adjacency matrix to a sparse edge index
        edge_index, _ = torch_geometric.utils.dense_to_sparse(dense_adj)

        edge_attribute = x.view(num_nodes*num_nodes, 1)

        d = Data(x=x, pos_edge_index=edge_index, edge_attr=edge_attribute, edge_index=edge_index)

        dataset.append(d)

    return dataset

def anti_vectorize_tensor(arr_2d, num_nodes=160, h=True):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch_size = arr_2d.shape[0]  # Extract the batch size
    adj_matrices = torch.zeros((batch_size, num_nodes, num_nodes)) # Create an array to store the adjacency matrices

    for i in range(batch_size):
        arr_1d = arr_2d[i].to(device)  # Extract the 1D array for this batch element
        adj_matrix = torch.zeros((num_nodes, num_nodes)).to(device)  # Create a matrix to store the adjacency matrix

        if h:
            cv, cb = h_i(num_nodes)
            adj_matrix[cv, cb] = arr_1d

        else:
            adj_matrix[np.triu_indices(n=num_nodes, k=1)] = arr_1d  # Assign the values to the upper triangular part
        adj_matrix_t = torch.transpose(adj_matrix,0,1)
        adj_matrix = (adj_matrix_t + adj_matrix)/2 # Add the transpose to make the matrix symmetric
        adj_matrix.fill_diagonal_(1)  # Set diagonal values to 1

        adj_matrices[i] = adj_matrix  # Add the adjacency matrix to the output array

    return adj_matrices

In [8]:
warnings.filterwarnings("ignore")
from torch.utils.data import Dataset
from torch_geometric.data import Batch,Data
from tqdm import tqdm
import wandb

class ListDataset(Dataset):
    def __init__(self, data_list):
        self.data = data_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        # Do any preprocessing you need on the sample here
        return sample

def collate_fn(batch):
    a = [x[0] for x in batch]
    b = [x[1] for x in batch]
    return [a,b]

def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False

def unfreeze_model(model):
    for param in model.parameters():
        param.requires_grad = True


In [9]:
aligner = Aligner()
generator = Generator()
discriminator = Discriminator()
# Losses
adversarial_loss1 = torch.nn.BCELoss()
l1_loss = torch.nn.L1Loss()

# send 1st GAN to GPU
aligner.to(device)
generator.to(device)
discriminator.to(device)
adversarial_loss1.to(device)
l1_loss.to(device)

print(torch.cuda.mem_get_info())

Aligner_optimizer = torch.optim.AdamW(aligner.parameters(), lr=0.025, betas=(0.5, 0.999))
generator_optimizer = torch.optim.AdamW(generator.parameters(), lr=0.025, betas=(0.5, 0.999))
discriminator_optimizer = torch.optim.AdamW(discriminator.parameters(), lr=0.025, betas=(0.5, 0.999))


Batch_Size = 1

X_casted_train_source = convert_func_data_to_graph_list(lr_train, N_SOURCE_NODES, h=False)
#X_casted_test_source = convert_func_data_to_graph_list(X_test_source, N_SOURCE_NODES, h=False)
X_casted_train_target = convert_func_data_to_graph_list(hr_train, N_TARGET_NODES, h=True)
#X_casted_test_target = convert_func_data_to_graph_list(X_test_target, N_TARGET_NODES, h=True)




(47334162432, 47622258688)


In [30]:
aligner.train()
generator.train()
discriminator.train()

nbre_epochs = N_EPOCHS



dataset = ListDataset(list(zip(X_casted_train_source, X_casted_train_target)))
#test_dataset = ListDataset(list(zip(X_casted_test_source, X_casted_test_target)))
train_loader = torch.utils.data.DataLoader(dataset, batch_size=Batch_Size, collate_fn=collate_fn, drop_last=False)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=Batch_Size, collate_fn=collate_fn, drop_last=False)


cv = KFold(n_splits=3, random_state=42, shuffle=True)
X, Y = lr_train, hr_train

for train_index, test_index in cv.split(X):
    X_casted_test_source = convert_func_data_to_graph_list(lr_train[test_index], N_SOURCE_NODES, h=False)
    X_casted_test_target = convert_func_data_to_graph_list(hr_train[test_index], N_TARGET_NODES, h=True)
    X_casted_train_source = convert_func_data_to_graph_list(lr_train[train_index], N_SOURCE_NODES, h=False)
    X_casted_train_target = convert_func_data_to_graph_list(hr_train[train_index], N_TARGET_NODES, h=True)
    dataset = ListDataset(list(zip(X_casted_train_source, X_casted_train_target)))
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=Batch_Size, collate_fn=collate_fn, drop_last=False)
    test_dataset = ListDataset(list(zip(X_casted_test_source, X_casted_test_target)))
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=Batch_Size, collate_fn=collate_fn, drop_last=False)

    for epochs in tqdm(range(nbre_epochs), desc="Epochs"):

        for i, x in tqdm(enumerate(train_loader), desc="Data_Loader"):

            Aligner_optimizer.zero_grad()
            generator_optimizer.zero_grad()
            discriminator_optimizer.zero_grad()

            data_source = x[0]
            data_target = x[1]

            cur_batch_size = len(data_source)

            data_batch_source = Batch.from_data_list(data_source).to(device)
            data_batch_target = Batch.from_data_list(data_target).to(device)


            A_output = aligner(data_batch_source)

            data_batch_source.x = A_output

            data_batch_source.edge_attr = A_output.view(cur_batch_size * N_SOURCE_NODES * N_SOURCE_NODES, 1)

            targett = data_batch_target.edge_attr.view(cur_batch_size, N_TARGET_NODES, N_TARGET_NODES)

            target_mean = torch.mean(targett, dim=(1, 2))
            target_std = torch.std(targett, dim=(1, 2))

            vectors = torch.empty(cur_batch_size, N_SOURCE_NODES_F)
            for j in range(cur_batch_size):
                vectors[j] = torch.normal(target_mean[j], target_std[j], size=(N_SOURCE_NODES_F,))

            d_target = vectors.to(device)

            target_d = anti_vectorize_tensor(d_target, num_nodes=N_SOURCE_NODES, h=False).to(device)

            kl_loss = Alignment_loss(target_d, A_output.view(cur_batch_size, N_SOURCE_NODES, N_SOURCE_NODES))


            ############################ Train Discriminator #################################################

            targett = data_batch_target.edge_attr.view(cur_batch_size, N_TARGET_NODES, N_TARGET_NODES)


            G_output = generator(data_batch_source)
            new_data = Data(x=G_output.view(cur_batch_size * N_TARGET_NODES, N_TARGET_NODES),
                            edge_index=data_batch_target.edge_index,
                            edge_attr=G_output.view(cur_batch_size * N_TARGET_NODES * N_TARGET_NODES, 1),
                            pos_edge_index=data_batch_target.edge_index)

            freeze_model(generator)
            freeze_model(aligner)
            unfreeze_model(discriminator)

            D_real = discriminator(data_batch_target)
            D_real_loss = adversarial_loss(D_real, (torch.ones_like(D_real)))
            D_fake = discriminator(new_data)
            D_fake_loss = adversarial_loss(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2


            D_loss.backward(retain_graph=True)
            discriminator_optimizer.step()


            unfreeze_model(generator)
            unfreeze_model(aligner)
            freeze_model(discriminator)

            Gg_loss = GT_loss(targett, G_output.view(cur_batch_size, N_TARGET_NODES, N_TARGET_NODES))


            D_fake = discriminator(new_data)

            D_fake = D_fake.view(-1)

            G_adversarial = adversarial_loss(D_fake, (torch.ones_like(D_fake)))

            G_loss = Gg_loss + G_adversarial + kl_loss


            G_loss.backward()

            generator_optimizer.step()

        print("[Epoch: %d]| [Ge loss: %f]| [D loss: %f]" % (
            epochs, G_loss, D_loss))

        aligner.eval()
        generator.eval()


        l1_l = []

        gen = []
        tar = []
        sou = []

        for it,x in tqdm(enumerate(test_loader),desc="Testing"):
            data_source = x[0]
            data_target = x[1]
            cur_batch_size = len(data_source)

            data_batch_source = Batch.from_data_list(data_source).to(device)

            sou.append(data_batch_source.x.view(cur_batch_size, N_SOURCE_NODES, N_SOURCE_NODES).detach().cpu())

            data_batch_target = Batch.from_data_list(data_target).to(device)
            A_test = aligner(data_batch_source)
            data_batch_source.x = A_test
            data_batch_source.edge_attr = A_test.view(cur_batch_size * N_SOURCE_NODES * N_SOURCE_NODES, 1)
            G_test = generator(data_batch_source)
            target = data_batch_target.edge_attr.view(cur_batch_size, N_TARGET_NODES, N_TARGET_NODES).detach().cpu()
            G_test = G_test.view(cur_batch_size, N_TARGET_NODES, N_TARGET_NODES).detach().cpu()
            G_test = post(G_test)
            l1 = F.l1_loss(G_test, target)
            l1_l.append(l1)

            gen.append(G_test.cpu())

            tar.append(target.cpu())


            G_test = G_test.numpy()
            target = target.numpy()

        final_l1 = np.mean(l1_l)

        final_gen = gen
        final_tar = tar
        final_sou = sou

        #with open(f"source_{fold_number + 1}.pkl", "wb") as f:
            #pickle.dump(final_sou, f)
            #f.close()

        #with open(f"target_{fold_number + 1}.pkl", "wb") as f:
            #pickle.dump(final_tar, f)
            #f.close()

        #with open(f"gen_{fold_number + 1}.pkl", "wb") as f:
            #pickle.dump(final_gen, f)
            #f.close()

        print(G_test[0])
        print(target[0])
        print(final_l1)


# #     ######################################### TESTING PART #########################################
      


Data_Loader: 0it [00:00, ?it/s]00:00<?, ?it/s]
Epochs:   0%|          | 0/10 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.09 GiB. GPU 0 has a total capacity of 44.35 GiB of which 3.07 GiB is free. Including non-PyTorch memory, this process has 41.27 GiB memory in use. Of the allocated memory 29.73 GiB is allocated by PyTorch, and 11.22 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [11]:
def post(x):

    for i in range(x.shape[1]):
        x[:, i, i] = 1

    x[x<0] = 0
    x[x>1] = 1

    x_tr = torch.transpose(x, 1, 2)
    x = (x + x_tr)/2

    return x



In [16]:
res = torch.empty((lr_test.shape[0], 268, 268))


X_casted_test_source = convert_func_data_to_graph_list(lr_test, N_SOURCE_NODES, h=False)

X_casted_test_target = convert_func_data_to_graph_list(lr_test, N_SOURCE_NODES, h=False)


test_dataset = ListDataset(list(zip(X_casted_test_source, X_casted_test_target)))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=Batch_Size, collate_fn=collate_fn, drop_last=False)



aligner.eval()
generator.eval()
gen = []

for it,x in tqdm(enumerate(test_loader),desc="Testing"):
    data_source = x[0]
    cur_batch_size = len(data_source)

    data_batch_source = Batch.from_data_list(data_source).to(device)

   
    A_test = aligner(data_batch_source)
    data_batch_source.x = A_test
    data_batch_source.edge_attr = A_test.view(cur_batch_size * N_SOURCE_NODES * N_SOURCE_NODES, 1)
    G_test = generator(data_batch_source)
    
    G_test = G_test.view(cur_batch_size, N_TARGET_NODES, N_TARGET_NODES).detach().cpu()
    G_test = post(G_test)
    

    res[it] = G_test.cpu()

Testing: 112it [00:12,  9.10it/s]


In [ ]:
 """ restore_aligner = "./weight" + "aligner_fold" + "_" + ".model"
        restore_generator = "./weight" + "generator_fold" + "_" + ".model"

        aligner.load_state_dict(torch.load(restore_aligner))
        generator.load_state_dict(torch.load(restore_generator))

        aligner.eval()
        generator.eval()

        i = 0
        predicted_test_graphs = []
        losses_test = []
        eigenvector_losses_test = []
        l1_tests = []
        Closeness_test = []
        Eigenvector_test = []
        for data_source, data_target in zip(X_casted_test_source, X_casted_test_target):
            # print(i)
            data_source_test = data_source.x.view(N_SOURCE_NODES, N_SOURCE_NODES)
            data_target_test = data_target.x.view(N_TARGET_NODES, N_TARGET_NODES)


            A_test = aligner(data_source)
            A_test_casted = convert_generated_to_graph_Al(A_test)
            A_test_casted = A_test_casted[0]
            data_target = data_target_test.detach().cpu().clone().numpy()
            # ************     Super-resolution    ************
            G_output_test = generator(A_test_casted)  # 35 x35
            G_output_test_casted = convert_generated_to_graph(G_output_test)
            G_output_test_casted = G_output_test_casted[0]
            torch.cuda.empty_cache()

            L1_test = l1_loss(data_target_test, G_output_test)
            # fold= 1
            target_test = data_target_test.detach().cpu().clone().numpy()
            predicted_test = G_output_test.detach().cpu().clone().numpy()
            source_test = data_source_test.detach().cpu().clone().numpy()

            torch.cuda.empty_cache()
            fake_topology_test = torch.tensor(topological_measures(predicted_test))
            real_topology_test = torch.tensor(topological_measures(target_test))

            eigenvector_test = (l1_loss(fake_topology_test[2], real_topology_test[2]))


            l1_tests.append(L1_test.detach().cpu().numpy())
            Eigenvector_test.append(eigenvector_test.detach().cpu().numpy())



        mean_l1 = np.mean(l1_tests)
        mean_eigenvector = np.mean(Eigenvector_test)

        # print("Mean L1 Loss Test: ", fold_mean_l1_loss)
        # print()

        losses_test.append(mean_l1)
        eigenvector_losses_test.append(mean_eigenvector) """"""

# fold += 1
#return (aligner, generator, discriminator)

In [23]:
from data_preparation import generate_submission_file
generate_submission_file(res, 'submission_files/trial_submission1.csv')

In [ ]:
#torch.save(aligner.state_dict(), "./weights/weight" + "aligner_fold" + f"_{fold_number}" + ".model")
#torch.save(generator.state_dict(), "./weights/weight" + "generator_fold" + f"_{fold_number}" + ".model")
#torch.save(discriminator.state_dict(), "./weights/weight" + "discriminator_fold" + f"_{fold_number}" + ".model")

####################################################TESTING PART####################################################

#restore_aligner = "./weights/weight" + "aligner_fold" + f"_{fold_number}" + ".model"
#restore_generator = "./weights/weight" + "generator_fold" + f"_{fold_number}" + ".model"
#aligner.load_state_dict(torch.load(restore_aligner, map_location=device))
#generator.load_state_dict(torch.load(restore_generator, map_location=device))


X_casted_test_source = convert_func_data_to_graph_list(lr_test, N_SOURCE_NODES, h=False)

X_casted_test_target = convert_func_data_to_graph_list(X_test_target, N_TARGET_NODES, h=True)


test_dataset = ListDataset(list(zip(X_casted_test_source, X_casted_test_target)))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=Batch_Size, collate_fn=collate_fn, drop_last=False)


aligner.eval()
generator.eval()


l1_loss = []

gen = []
tar = []
sou = []

for it,x in tqdm(enumerate(test_loader),desc="Testing"):
    data_source = x[0]
    data_target = x[1]
    cur_batch_size = len(data_source)

    data_batch_source = Batch.from_data_list(data_source).to(device)

    sou.append(data_batch_source.x.view(cur_batch_size, N_SOURCE_NODES, N_SOURCE_NODES).detach().cpu())

    data_batch_target = Batch.from_data_list(data_target).to(device)
    A_test = aligner(data_batch_source)
    data_batch_source.x = A_test
    data_batch_source.edge_attr = A_test.view(cur_batch_size * N_SOURCE_NODES * N_SOURCE_NODES, 1)
    G_test = generator(data_batch_source)
    target = data_batch_target.edge_attr.view(cur_batch_size, N_TARGET_NODES, N_TARGET_NODES).detach().cpu()
    G_test = G_test.view(cur_batch_size, N_TARGET_NODES, N_TARGET_NODES).detach().cpu()
    G_test = post(G_test)
    l1 = F.l1_loss(G_test, target)
    l1_loss.append(l1)

    gen.append(G_test.cpu())

    tar.append(target.cpu())


    G_test = G_test.numpy()
    target = target.numpy()

final_l1 = np.mean(l1_loss)

final_gen = gen
final_tar = tar
final_sou = sou

with open(f"source_{fold_number + 1}.pkl", "wb") as f:
    pickle.dump(final_sou, f)
    f.close()

with open(f"target_{fold_number + 1}.pkl", "wb") as f:
    pickle.dump(final_tar, f)
    f.close()

with open(f"gen_{fold_number + 1}.pkl", "wb") as f:
    pickle.dump(final_gen, f)
    f.close()

print(G_test[0])
print(target[0])
print(final_l1)

In [50]:
warnings.filterwarnings("ignore")


"""#Training"""

torch.cuda.empty_cache()
torch.cuda.empty_cache()

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("running on GPU")
else:
    device = torch.device("cpu")
    print("running on CPU")

source_data = np.random.normal(0, 0.5, (N_SUBJECTS, N_SOURCE_NODES_F))
target_data = np.random.normal(0, 0.5, (N_SUBJECTS, N_TARGET_NODES_F))

kf = KFold(n_splits=3, shuffle=True, random_state=1773)

fold = 0
losses_test = []
closeness_losses_test = []
# betweenness_losses_test = []
eigenvector_losses_test = []

#for train_index, test_index in kf.split(source_data):
    # print( * "#" + " FOLD " + str(fold) + " " +  * "#")
    #X_train_source, X_test_source, X_train_target, X_test_target = source_data[train_index], source_data[test_index], target_data[train_index], target_data[test_index]
   
aligner, generator, discriminator = IMANGraphNet(lr_train,  hr_train, 1)




test_mean = np.mean(l1_test)
Eigenvector_test_mean = np.mean(eigenvector_test)
plot_source(source_test)
plot_target(data_target)
plot_target(predicted_test)

print("Mean L1 Test", test_mean)

print("Mean Eigenvector Test", Eigenvector_test_mean)


running on GPU


Data_Loader: 0it [00:00, ?it/s]00:00<?, ?it/s]
Epochs:   0%|          | 0/10 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.44 GiB. GPU 0 has a total capacity of 15.70 GiB of which 1.10 GiB is free. Process 204151 has 5.68 GiB memory in use. Process 210271 has 3.73 GiB memory in use. Process 214347 has 4.05 GiB memory in use. Including non-PyTorch memory, this process has 896.00 MiB memory in use. Of the allocated memory 615.99 MiB is allocated by PyTorch, and 8.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)